In [1]:
import pandas as pd
import json

In [2]:
i = 0
with open("passwords1.txt") as infile:
    for line in infile:
        i = i+1
print('Number of passwords in passwords1.txt', i)

Number of passwords in passwords1.txt 100000000


In [20]:
i = 0
with open("passwords2.txt") as infile:
    for line in infile:
        i = i+1
print('Number of passwords in passwords2.txt', i)

Number of passwords in passwords2.txt 39000000


In [9]:
ord('a')

97

In [8]:
pip install sympy

Note: you may need to restart the kernel to use updated packages.


We choose m(dimension of the future bit array) in respect of a 7% false positive rate that we will pose in the future bloom filter, so the module of the hash fucntions will the the first greater prime of 553490135.

In [10]:
import sympy as s
m = s.nextprime(553490135)
m

553490137

In [3]:
import random
random.randrange(m)

212148138

We will create 4 hash functions, that we will later describe, the first thing we that I need 4 vectors with 20 components, all randomly generated in between 0 and m.

In [4]:
def randomizr(m): 
    vec = []
    for i in range(20):
        vec.append(random.randrange(m))
    return vec


In [5]:
vector1 = randomizr(m)
vector2 = randomizr(m)
vector3 = randomizr(m)
vector4 = randomizr(m)
print(vector1,'\n',vector2, '\n', vector3, '\n', vector4)
print(vector1 == vector2 or vector1==vector3 or vector1==vector4 or vector2 == vector3 or vector3==vector4)

[320032415, 124596468, 192243248, 210600885, 469279337, 363944069, 428720719, 464995414, 142099927, 100010091, 189760839, 292309987, 191899598, 434784156, 9828227, 438433839, 379787659, 511525427, 383277022, 207476111] 
 [196878839, 414628127, 121047087, 88914908, 506086479, 509913332, 442529809, 92785850, 276855162, 78990086, 208974229, 231678954, 273784351, 206209961, 441568291, 551861911, 360218731, 244146914, 483924221, 482762323] 
 [414795905, 399707943, 519985065, 91158296, 123533042, 4699813, 281354659, 306748969, 3414012, 521779425, 434949211, 117222902, 45998078, 81912791, 161085663, 181661559, 423762748, 192380751, 189679520, 321212085] 
 [378547270, 152879104, 193866808, 82129304, 437351035, 307560657, 98108298, 353643679, 406975444, 273737614, 258928089, 432601128, 53903005, 397466955, 326740583, 63708514, 236761772, 196259849, 294424851, 551195740]
False


Now that the vectors are all different, to avoid changing them every time we run the file, we save the vectors with the values printed above. We did that to make explicit the uniform distribution to chooes those components.

In [2]:
vector1 = [320032415, 124596468, 192243248, 210600885, 469279337, 363944069, 428720719, 464995414, 142099927, 100010091, 189760839, 292309987, 191899598, 434784156, 9828227, 438433839, 379787659, 511525427, 383277022, 207476111] 
vector2 = [196878839, 414628127, 121047087, 88914908, 506086479, 509913332, 442529809, 92785850, 276855162, 78990086, 208974229, 231678954, 273784351, 206209961, 441568291, 551861911, 360218731, 244146914, 483924221, 482762323]
vector3 = [414795905, 399707943, 519985065, 91158296, 123533042, 4699813, 281354659, 306748969, 3414012, 521779425, 434949211, 117222902, 45998078, 81912791, 161085663, 181661559, 423762748, 192380751, 189679520, 321212085]
vector4 = [378547270, 152879104, 193866808, 82129304, 437351035, 307560657, 98108298, 353643679, 406975444, 273737614, 258928089, 432601128, 53903005, 397466955, 326740583, 63708514, 236761772, 196259849, 294424851, 551195740]


In [21]:
#vector1 = [548373287, 222232648, 393628080, 220566875, 315011190, 457014546, 401445075, 165870502, 544231662, 431020630, 189088991, 382246549, 207652822, 296995918, 282393338, 361281726, 379519948, 289192586, 422903918, 179727888]
#vector2 = [510338572, 428859708, 127662473, 411969966, 268400108, 235368761, 316744780, 506406582, 545941917, 173198424, 519259740, 487278310, 165823038, 452155477, 51364823, 418169255, 277375778, 416769453, 246767791, 492198197]
#vector3 = [372907205, 539742227, 324783969, 297544716, 334245771, 88148016, 531791743, 323977773, 386017264, 22958949, 341277333, 215029602, 313390962, 343400267, 91334501, 510518004, 437634524, 319955216, 323354422, 493771037]
#vector4 = [429698059, 384014737, 257859287, 93358385, 410243180, 386311450, 514566367, 450082373, 480763392, 117581539, 403105809, 121520178, 313661122, 13845469, 325510496, 256279553, 371541715, 133092061, 56891456, 90041203]
#done!

In [22]:
#import numpy as np
#def simple_dot(a,b):
   # return(np.dot(a,b)%m)

In [3]:
def simple_dot(x,y):
    k = 0
    A = 0
    for i in range(20):
        #print(x[i])
        A = A +((x[i])*y[i])%m
    return(A%m)

Now we define the 4 hash fucntions, that will actually be the same but with different coefficients given by the vectors we just defined.

In [4]:
def Hash1(x): 
    vec = []
    for i in x:
        vec.append(ord(i))
    return (simple_dot(vec,vector1))


In [5]:
def Hash2(x): 
    vec = []
    for i in x:
        vec.append(ord(i))
    return (simple_dot(vec,vector2))

In [6]:
def Hash3(x): 
    vec = []
    for i in x:
        vec.append(ord(i))
    return (simple_dot(vec,vector3))

In [7]:
def Hash4(x): 
    vec = []
    for i in x:
        vec.append(ord(i))
    return (simple_dot(vec,vector4))

In [14]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

Now that I have 4 Hash functions I need to build the bloom filter!

In [24]:
import time
def BloomFilter3(passwords1, passwords2):
    start = time.time()
    filtr = [0]*553490137 #here m. the module
    probably = 0
    n = 0
    k = 0
    #first i put in my filter all the password from passwords1, signing with a 1 the components where there is something
    with open(passwords1) as infile:
        for line in infile:
            item = line.replace('\n','')
            #print(item)
            A = Hash1(item)
            B = Hash2(item)
            C = Hash3(item)
            D = Hash4(item)
            #print(A,B,C,D)
            filtr[A] = 1
            filtr[B] = 1
            filtr[C] = 1
            filtr[D] = 1
            n = n+1
            if n == 20000000:
                break
    #now that every password from df passwords1 is inside, we have to put the new ones and count homw many are probably duplicates
    with open(passwords2) as infile:
        for line in infile:
            item = line.replace('\n','')
            #print(filtr)
            #print(item)
            #print(filtr[Hash1(item)])
            if filtr[Hash1(item)] == 1:
                if filtr[Hash2(item)] == 1:
                    if filtr[Hash3(item)] == 1:
                        if filtr[Hash4(item)] == 1:
                            probably = probably + 1
            k = k+1
            if k == 20950000:
                break
    end = time.time()
    #for the probability of false positive I build the model from the passwords1 size and i false positive rate i wanted, so it's 7%
    print('Number of hash function used: ', 4)
    print('Number of duplicates detected: ', probably)
    print('Probability of false positives: ', 0.07)
    print('Execution time: ', end-start)
    return 

In [25]:
BloomFilter3('passwords1.txt', 'passwords2.txt')

KeyboardInterrupt: 

13.16

Now we will modify a little the bloom filter to asnwer the bonus question about the exact number of false positives.

In [47]:
import time
def BloomFilter2(passwords1, passwords2):
    n = 0
    k = 0
    start = time.time()
    filtr = [0]*m #here m. the module
    false_pos = 0
    #first i put in my filter all the password from passwords1, signing with a 1 the components where there is something
    dic_hash1 = {}
    with open(passwords1) as infile:
        for line in infile:
            item = line.replace('\n','')
            A = Hash1(item)
            B = Hash2(item)
            C = Hash3(item)
            D = Hash4(item)
            filtr[A] = 1
            try:
                dic_hash1[A].append(item)
            except KeyError:
                dic_hash1[A] = [item]
            filtr[B] = 1
            filtr[C] = 1
            filtr[D] = 1
            n = n+1
            if n == 1000000:
                break
    #now that every password from df passwords1 is inside, we have to put the new ones and count homw many are probably duplicates
    with open(passwords2) as infile:
        for line in infile:
            item = line.replace('\n','')
            A = Hash1(item)
            if filtr[Hash1(item)] == 1:
                if filtr[Hash2(item)] == 1:
                    if filtr[Hash3(item)] == 1:
                        if filtr[Hash4(item)] == 1:
                            if item not in dic_hash1[A]:
                                false_pos = false_pos + 1
            k = k+1
            if k == 390000:
                break
    end = time.time()
    #for the probability of false positive I build the model from the passwords1 size and i false positive rate i wanted, so it's 7%
    print('Number of hash function used: ', 4)
    print('Number of false positives: ', false_pos)
    print('Theoretical probability of false positives: ', 0.07)
    print('Actual false positives probability', false_pos/300000)
    print('Execution time: ', end-start)
    return 

In [48]:
BloomFilter2('passwords1.txt', 'passwords2.txt')

Number of hash function used:  4
Number of false positives:  0
Theoretical probability of false positives:  0.07
Actual false positives probability 0.0
Execution time:  236.54372000694275
